# Analyze NASDAQ ITCH protocol

In [2]:
import gzip
import shutil
from pathlib import Path
from urllib.request import urlretrieve
from urllib.parse import urljoin
from datetime import datetime
from struct import unpack
from collections import namedtuple, Counter
from datetime import timedelta
from time import time

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## Download Sample Data

In [ ]:
URL = 'https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/'
SOURCE_FILE = '03272019.NASDAQ_ITCH50.gz' # .gz 5.13GB, unzipped 11.99GB
data_path = Path('data')

url = urljoin(URL, SOURCE_FILE)
filename = data_path / url.split('/')[-1]
if not data_path.exists():
    print('Creating directory')
    data_path.mkdir()
if not filename.exists():
    print('Downloading...', url)
    urlretrieve(url, filename)
unzipped = data_path / (filename.stem + '.bin')
if not (data_path / unzipped).exists():
    print('Unzipping to', unzipped)
    with gzip.open(str(filename), 'rb') as f_in:
        with open(unzipped, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)


Creating directory
Downloading... https://emi.nasdaq.com/ITCH/Nasdaq%20ITCH/03272019.NASDAQ_ITCH50.gz
Unzipping to data/03272019.NASDAQ_ITCH50.bin


## Message Formats

In [ ]:
# https://www.nasdaqtrader.com/content/technicalsupport/specifications/dataproducts/NQTVITCHSpecification.pdf

In [1]:
event_codes = {'O': 'Start of Messages',
               'S': 'Start of System Hours',
               'Q': 'Start of Market Hours',
               'M': 'End of Market Hours',
               'E': 'End of System Hours',
               'C': 'End of Messages'}

encoding = {'primary_market_maker': {'Y': 1, 'N': 0},
            'printable'           : {'Y': 1, 'N': 0},
            'buy_sell_indicator'  : {'B': 1, 'S': -1},
            'cross_type'          : {'O': 0, 'C': 1, 'H': 2},
            'imbalance_direction' : {'B': 0, 'S': 1, 'N': 0, 'O': -1}}

formats = {
    ('integer', 2): 'H',
    ('integer', 4): 'I',
    ('integer', 6): '6s',
    ('integer', 8): 'Q',
    ('alpha', 1)  : 's',
    ('alpha', 2)  : '2s',
    ('alpha', 4)  : '4s',
    ('alpha', 8)  : '8s',
    ('price_4', 4): 'I',
    ('price_8', 8): 'Q',
}
